# Phase 5.2 - Kaggle End-to-End API Testing

This notebook tests the complete Pharma-Safe Lens backend API on Kaggle.

## Setup Requirements
- Kaggle Internet enabled
- Python 3.10+
- GPU not required (CPU-only testing)

## Step 1: Install System Dependencies

In [ ]:
# Install Tesseract OCR (required for pytesseract)
!apt-get update -y
!apt-get install -y tesseract-ocr
!tesseract --version

## Step 2: Clone Repository

In [ ]:
# Clone the repository
!git clone https://github.com/YOUR_USERNAME/pharma-safe-lens.git
%cd pharma-safe-lens/backend

## Step 3: Install Python Dependencies

In [ ]:
# Install backend dependencies
!pip install -r requirements.txt

## Step 4: Start FastAPI Server in Background

In [ ]:
import subprocess
import time
import sys

# Add parent directory to path
sys.path.insert(0, '/kaggle/working/pharma-safe-lens')

# Start FastAPI server in background
server_process = subprocess.Popen(
    ['uvicorn', 'backend.app.main:app', '--host', '0.0.0.0', '--port', '8000'],
    cwd='/kaggle/working/pharma-safe-lens',
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
time.sleep(5)
print("✅ FastAPI server started on port 8000")

## Step 5: Test Health Check Endpoint

In [ ]:
import requests
import json

# Test root endpoint
response = requests.get('http://localhost:8000/')
print("Root Endpoint:")
print(json.dumps(response.json(), indent=2))

# Test health check
response = requests.get('http://localhost:8000/health')
print("\nHealth Check:")
print(json.dumps(response.json(), indent=2))

## Step 6: Create Test Image

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import io

# Create a test image with drug names
img = Image.new('RGB', (600, 300), color='white')
draw = ImageDraw.Draw(img)

# Use default font (arial may not be available on Kaggle)
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 48)
except:
    font = ImageFont.load_default()

# Draw drug names that have known interactions
draw.text((30, 80), "ASPIRIN 100MG", fill='black', font=font)
draw.text((30, 180), "WARFARIN 5MG", fill='black', font=font)

# Save image
img.save('test_medications.png')
print("✅ Test image created: test_medications.png")

# Display the image
img

## Step 7: Test /analyze-image Endpoint

In [ ]:
import time

# Open the test image
with open('test_medications.png', 'rb') as f:
    files = {'file': ('test_medications.png', f, 'image/png')}
    
    # Record start time
    start_time = time.time()
    
    # Send request
    response = requests.post(
        'http://localhost:8000/api/v1/analyze-image',
        files=files
    )
    
    # Record end time
   end_time = time.time()
    latency = end_time - start_time

print(f"🕐 Response Time: {latency:.2f} seconds")
print(f"📊 Status Code: {response.status_code}")
print("\n" + "="*70)
print("RESPONSE:")
print("="*70)
print(json.dumps(response.json(), indent=2))

## Step 8: Validate Response Structure

In [ ]:
# Parse response
data = response.json()

print("VALIDATION CHECKS:")
print("="*70)

# Check required fields
checks = [
    ('status' in data, "✅ Has 'status' field" if 'status' in data else "❌ Missing 'status'"),
    ('detected_drugs' in data, "✅ Has 'detected_drugs' field" if 'detected_drugs' in data else "❌ Missing 'detected_drugs'"),
    ('interactions' in data, "✅ Has 'interactions' field" if 'interactions' in data else "❌ Missing 'interactions'"),
    (data.get('status') == 'success', "✅ Status is 'success'" if data.get('status') == 'success' else f"❌ Status is '{data.get('status')}'"),
    (len(data.get('detected_drugs', [])) >= 2, f"✅ Detected {len(data.get('detected_drugs', []))} drugs" if len(data.get('detected_drugs', [])) >= 2 else f"❌ Only detected {len(data.get('detected_drugs', []))} drug(s)"),
    (len(data.get('interactions', [])) > 0, f"✅ Found {len(data.get('interactions', []))} interaction(s)" if len(data.get('interactions', [])) > 0 else "❌ No interactions found"),
]

for passed, message in checks:
    print(message)

# Detailed interaction analysis
if data.get('interactions'):
    print("\n" + "="*70)
    print("INTERACTION DETAILS:")
    print("="*70)
    for i, interaction in enumerate(data['interactions'], 1):
        print(f"\nInteraction {i}:")
        print(f"  Drug Pair: {interaction.get('drug_pair')}")
        print(f"  Risk Level: {interaction.get('risk_level')}")
        print(f"  Clinical Effect: {interaction.get('clinical_effect', '')[:100]}...")
        print(f"  Safety Alert: {interaction.get('safety_alert')}")
        print(f"  AI Explanation Length: {len(interaction.get('ai_explanation', ''))} chars")

## Step 9: Performance Benchmarking

In [ ]:
import numpy as np

print("PERFORMANCE BENCHMARKING")
print("="*70)
print("Running 10 requests to measure average latency...\n")

latencies = []

for i in range(10):
    with open('test_medications.png', 'rb') as f:
        files = {'file': ('test_medications.png', f, 'image/png')}
        
        start = time.time()
        response = requests.post('http://localhost:8000/api/v1/analyze-image', files=files)
        end = time.time()
        
        latency = end - start
        latencies.append(latency)
        print(f"Request {i+1}: {latency:.3f}s")

print("\n" + "="*70)
print(f"Average Latency: {np.mean(latencies):.3f}s")
print(f"Min Latency: {np.min(latencies):.3f}s")
print(f"Max Latency: {np.max(latencies):.3f}s")
print(f"Std Deviation: {np.std(latencies):.3f}s")
print("="*70)

## Step 10: Test Error Handling

In [ ]:
print("ERROR HANDLING TESTS")
print("="*70)

# Test 1: Empty image
print("\nTest 1: Blank image (no text)")
blank_img = Image.new('RGB', (200, 200), color='blue')
img_bytes = io.BytesIO()
blank_img.save(img_bytes, format='PNG')
img_bytes.seek(0)

response = requests.post(
    'http://localhost:8000/api/v1/analyze-image',
    files={'file': ('blank.png', img_bytes, 'image/png')}
)
print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

# Test 2: Single drug (no interaction)
print("\nTest 2: Single drug image")
single_img = Image.new('RGB', (400, 150), color='white')
draw = ImageDraw.Draw(single_img)
try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 48)
except:
    font = ImageFont.load_default()
draw.text((30, 50), "METFORMIN 500MG", fill='black', font=font)

img_bytes = io.BytesIO()
single_img.save(img_bytes, format='PNG')
img_bytes.seek(0)

response = requests.post(
    'http://localhost:8000/api/v1/analyze-image',
    files={'file': ('single.png', img_bytes, 'image/png')}
)
print(f"Status: {response.status_code}")
print(f"Response: {json.dumps(response.json(), indent=2)}")

## Step 11: Cleanup

In [ ]:
# Stop the server
server_process.terminate()
server_process.wait()
print("✅ Server stopped")

## Summary

### Phase 5.2 Validation Checklist:

- [ ] FastAPI server starts successfully
- [ ] Health check endpoint works
- [ ] /analyze-image endpoint processes images
- [ ] OCR extracts drug names correctly
- [ ] Drug normalization works
- [ ] Interaction checking is accurate
- [ ] AI explanations are generated
- [ ] Safety validation blocks dangerous text
- [ ] Response structure is correct
- [ ] Latency is acceptable (< 5 seconds per request)
- [ ] Error handling works for edge cases

### Next Steps:

Once all checks pass:
1. Document API endpoint specifications
2. Create API usage examples
3. Proceed to Phase 6 - Frontend Development